In [1]:
%load_ext autoreload

In [32]:
%autoreload 2
from collections import defaultdict
from datetime import datetime
import h5py
import matplotlib.pyplot as plt
from neural_analysis.matIO import loadmat
import numpy as np
import os
import pandas as pd
from scipy.signal import butter, filtfilt, freqz
from scipy.stats import pearsonr
import shutil
import sys
import time
from tqdm.auto import tqdm

sys.path.append('../')
from ld_utils import compute_eigs, get_data_sigma_norms
from up_down import get_up_down
from utils import get_binary_stimuli, get_sample_interval, load, save, to_string

## Load Neural Data and Split Into Segments

In [4]:
session = 'Mary-Anesthesia-20160809-01'
# session = 'MrJones-Anesthesia-20160113-01.mat'
# filename = fr'/home/adameisen/millerdata/common/datasets/anesthesia/mat/propofolPuffTone/{session}.mat'
filename = fr'/om/user/eisenaj/ChaoticConsciousness/data/propofolPuffTone/{session}.mat'
print("Loading data ...")
start = time.process_time()
electrode_info, lfp, lfp_schema, session_info, spike_times, unit_info = loadmat(filename, variables=['electrodeInfo', 'lfp', 'lfpSchema', 'sessionInfo', 'spikeTimes', 'unitInfo'], verbose=False)
spike_times = spike_times[0]
dt = lfp_schema['smpInterval'][0]
T = lfp.shape[0]

# f = h5py.File(filename, 'r')
# airPuff_binary, audio_binary = get_binary_stimuli(f)

print(f"Data loaded (took {time.process_time() - start:.2f} seconds)")

Loading data ...
Data loaded (took 129.32 seconds)


In [11]:
end_step = T - int(30/dt) # take off 30 seconds
data = lfp[:end_step]
for window in [2.5]:
    stride = 2.5
      
    data_dir = f"/om/user/eisenaj/ChaoticConsciousness/data/propofolPuffTone/{session}_window_{window}_stride_{stride}"
    os.makedirs(data_path, exist_ok=True)
    
    print(f"window = {window} s")
    num_windows = int(end_step/int(stride/dt))
    for i in tqdm(range(num_windows)):
        start = int(stride/dt)*i
        end = int(stride/dt)*i + int(window/dt)
        window_data = data[start:end]
        
        window_dict = {'data': window_data, 'start_ind': start, 'start_time': start*dt, 'end_ind': end, 'end_time': end*dt}
        save(window_dict, os.path.join(data_dir, f"window_{i}"))
        
#         print(f"start = {start}, end = {end}")

window = 2.5 s


  0%|          | 0/2822 [00:00<?, ?it/s]

## Compile Results and Save

In [13]:
folder_to_compile = "/om/user/eisenaj/ChaoticConsciousness/results/propofol/correlations/correlations_Mary-Anesthesia-20160809-01_window_2.5_stride_2.5"
assert os.path.exists(folder_to_compile)

In [16]:
files = os.listdir(folder_to_compile)

all_window_data = []
for file in tqdm(files):
    window_data = load(os.path.join(folder_to_compile, file))
    del window_data['data']
    all_window_data.append(window_data) 

  0%|          | 0/2822 [00:00<?, ?it/s]

In [29]:
results_df = pd.DataFrame(all_window_data)

In [30]:
results_df = results_df.sort_values('start_ind')
results_df = results_df.reset_index(drop=True)

In [33]:
shutil.rmtree(folder_to_compile)

In [34]:
save(results_df, folder_to_compile)